<a href="https://colab.research.google.com/github/amelft81/ASDEEG/blob/main/Combine_2D_Band_Power_Maps_to_RGB_Image.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def normalize_band_to_uint8(arr):
    """
    Normalizes a 2D array to the 0-255 range and converts it to uint8.
    Handles cases where min/max might be the same (flat array).
    """
    min_val = np.min(arr)
    max_val = np.max(arr)
    if max_val == min_val:
        # If all values are the same, return a grey image (e.g., all 128)
        return np.full(arr.shape, 128, dtype=np.uint8)
    return ((arr - min_val) / (max_val - min_val) * 255).astype(np.uint8)

def combine_band_maps_to_rgb(theta_map, alpha_map, beta_map):
    """
    Combines three 2D numpy arrays (representing Theta, Alpha, Beta band power maps)
    into a single 3-channel (RGB) image.

    Args:
        theta_map (np.ndarray): 2D array for Theta band power.
        alpha_map (np.ndarray): 2D array for Alpha band power.
        beta_map (np.ndarray): 2D array for Beta band power.

    Returns:
        np.ndarray: A 3D numpy array (height, width, 3) representing the RGB image.
                    The pixel values are in uint8 (0-255).
    """
    # Ensure all input maps have the same dimensions
    if not (theta_map.shape == alpha_map.shape == beta_map.shape):
        raise ValueError("All input band maps must have the same dimensions.")

    # Normalize each band map to 0-255 range and convert to uint8
    # Theta -> Red channel
    # Alpha -> Green channel
    # Beta  -> Blue channel
    img_r = normalize_band_to_uint8(theta_map)
    img_g = normalize_band_to_uint8(alpha_map)
    img_b = normalize_band_to_uint8(beta_map)

    # Stack the normalized channels to create the RGB image
    rgb_image = np.stack([img_r, img_g, img_b], axis=-1)

    return rgb_image

if __name__ == "__main__":
    # --- Example Usage ---
    # Simulate three 2D interpolated band power maps
    # In a real scenario, these would come from your EEG signal processing
    # and interpolation steps, similar to how they are generated in the
    # `create_eeg_image` function of the `eeg_dataset_generator` artifact.

    image_dim = 224 # Example dimension, matching the paper's image size

    # Create synthetic data for demonstration
    # Imagine these are the interpolated power values across the scalp
    # We'll use simple gradients and patterns for visualization
    x = np.linspace(0, 1, image_dim)
    y = np.linspace(0, 1, image_dim)
    X, Y = np.meshgrid(x, y)

    # Simulate Theta map (e.g., higher power in top-left)
    simulated_theta_map = np.sin(X * 5) + np.cos(Y * 5) + np.random.rand(image_dim, image_dim) * 0.1

    # Simulate Alpha map (e.g., circular pattern)
    center_x, center_y = 0.5, 0.5
    radius = np.sqrt((X - center_x)**2 + (Y - center_y)**2)
    simulated_alpha_map = np.exp(-10 * radius**2) + np.random.rand(image_dim, image_dim) * 0.1

    # Simulate Beta map (e.g., diagonal gradient)
    simulated_beta_map = X + Y + np.random.rand(image_dim, image_dim) * 0.1

    print("Simulated 2D band power maps created.")
    print(f"Shape of each map: {simulated_theta_map.shape}")

    # Combine the simulated maps into an RGB image
    rgb_eeg_image = combine_band_maps_to_rgb(
        simulated_theta_map,
        simulated_alpha_map,
        simulated_beta_map
    )

    print(f"\nCombined RGB image shape: {rgb_eeg_image.shape}")
    print(f"Data type of RGB image: {rgb_eeg_image.dtype}")

    # Display the resulting RGB image
    plt.figure(figsize=(7, 7))
    plt.imshow(rgb_eeg_image)
    plt.title("Combined EEG Band Power Map (RGB)")
    plt.axis('off')
    plt.show()

    print("\nSuccessfully combined 2D band maps into an RGB image and displayed it.")